# **Name - Prabhakar Kumar**
# **Roll - IRM2017008**
# **Assignment 4 Question 2**

In [1]:
import numpy as np
import pandas as pd
import csv

# **Loading the Dataset**

In [14]:
data = pd.DataFrame(pd.read_csv('Microchip.csv',delimiter=','))
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118 entries, 0 to 117
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X1      118 non-null    float64
 1   X2      118 non-null    float64
 2   Y       118 non-null    int64  
dtypes: float64(2), int64(1)
memory usage: 2.9 KB


In [15]:
data.head()

,X1,X2,Y
0,0.051267,0.69956,1
1,-0.092742,0.68494,1
2,-0.213710,0.69225,1
3,-0.375000,0.50219,1
4,-0.513250,0.46564,1


In [16]:
x = data[['X1','X2']].values
y = data['Y'].values

# **Applying Box Muller Transformation**

In [18]:
np.seterr(divide = 'ignore')
def gauss_func(distribution1,distribution2):
  normal_1=np.sqrt(-2*np.log(distribution1))*np.cos(2*np.pi*distribution2)
  normal_2=np.sqrt(-2*np.log(distribution1))*np.sin(2*np.pi*distribution2)
  return normal_1,normal_2

In [20]:
x0=[]
x1=[]
min_x0=x[0][0]
max_x0=x[i][0]
min_x1=x[i][1]
max_x1=x[i][1]

n=y.shape[0]

for i in range(n):
  x0.append(x[i][0])
  x1.append(x[i][1])
  if(x[i][0]>max_x0):
    max_x0=x[i][0]
  if(x[i][0]<min_x0):
    min_x0=x[i][0]
  if(x[i][1]>max_x1):
    max_x1=x[i][1]
  if(x[i][1]<min_x1):
    min_x1=x[i][1]

for i in range(n):
  x0[i]=(x0[i]-min_x0)/(max_x0-min_x0)
  x1[i]=(x1[i]-min_x1)/(max_x1-min_x1)

x0=np.array(x0)
x1=np.array(x1)

print("Here")

norm1,norm2 = gauss_func(x0,x1)

Here


# **Splitting Dataset into Training and Test data**

In [21]:
import random
n=y.shape[0]
temp_list=[]
for i in range(n):
  temp_list.append(i)


random.shuffle(temp_list)
train_limit=int(0.7*n)

temp_X=[]
temp_Y=[]
temp_test_X=[]
temp_test_Y=[]
for i in range(n):
  x0=x[temp_list[i]][0]
  x1=x[temp_list[i]][1]
  if(i<=train_limit):
    temp_append=x[temp_list[i],:]
    temp_X.append(temp_append)
    temp_Y.append(y[temp_list[i]])
  else:
    temp_append=x[temp_list[i],:]
    temp_test_X.append(temp_append)
    temp_test_Y.append(y[temp_list[i]])


In [22]:
X=np.array(temp_X)
Y=np.array(temp_Y)
X_test=np.array(temp_test_X)
Y_test=np.array(temp_test_Y)

In [23]:
print(X.shape)
print(X_test.shape)

(83, 2)
(35, 2)


In [24]:
type(X)

numpy.ndarray

# **Calculation of Phi Parameter**

In [25]:
def get_phi():
  n = Y.shape[0]
  global count_0
  global count_1
  for i in range(n):
    if(Y[i]==1):
      count_1+=1
    else:
      count_0+=1
  return count_1/n

count_0=0
count_1=0
phi = get_phi()
print(phi)

0.4939759036144578


# **Mean Parameter for Class 0**

In [26]:
def get_mu0():
  global count_0
  n=Y.shape[0]
  sum=np.zeros_like(X[0,:])
  for i in range(n):
    if(Y[i]==1):
      continue
    sum=sum+X[i,:]
  print(count_0)
  print(sum)
  ret=sum/count_0
  return ret

mu0 = get_mu0()
print(mu0)

42
[ 4.7795896 10.895514 ]
[0.11379975 0.259417  ]


# **Mean Parameter for Class 1**

In [27]:
def get_mu1():
  global count_1
  n=Y.shape[0]
  sum=np.zeros_like(X[0,:])
  for i in range(n):
    if(Y[i]==0):
      continue
    sum=sum+X[i,:]
  print(count_1)
  print(sum)
  ret=sum/count_1
  return ret

mu1 = get_mu1()
print(mu1)

41
[1.8427446 6.781388 ]
[0.04494499 0.16539971]


# **Calculation of Sigma**

In [28]:
def get_sigma():
  global mu0
  global mu1
  n=Y.shape[0]
  sum = np.zeros((X.shape[1],X.shape[1]))
  print(sum)
  for i in range(n):
    xi = X[i,:]
    xi = xi.reshape((-1,1))
    yi = Y[i]
    if(yi==0):
      temp_sum = ((xi-mu0).T)*(xi-mu0)
    else:
      temp_sum = ((xi-mu1).T)*(xi-mu1)
    sum = sum + temp_sum
  return sum/n

sigma = get_sigma()
print(sigma)


[[0. 0.]
 [0. 0.]]
[[ 0.23085504 -0.06132183]
 [-0.06132183  0.28786302]]


# **Prediction over Test Data**

In [29]:
def get_p_of_x_given_y(x, mu):
  pi = 3.14
  dim = mu.shape[0]
  #exp(-0.5*(x-mu)'*inv(sigma)*(x-mu))
  temp1=np.exp(-0.5*((x-mu).T)*np.linalg.inv(sigma)*(x-mu))
  #np.linalg.det(a)
  temp2=(1/(2*pi)**(dim/2))*np.sqrt(np.linalg.det(sigma))
  ret=temp1*temp2
  return ret[0]

In [30]:
def get_post_prob(y):
  global phi
  return (phi**y)*((1-phi)**(1-y))

n=Y_test.shape[0]

tp=0
tn=0
fp=0
fn=0

print("Total = ",n)
for index in range(n):
  temp_prob=get_post_prob(Y_test[index])
  p0=get_p_of_x_given_y(X_test[index,:],mu0)
  p1=get_p_of_x_given_y(X_test[index,:],mu1)

  prob0 = p0[0]*temp_prob
  prob1 = p1[0]*temp_prob
  ans=1
  if(prob0>prob1):
    ans=0
  else:
    ans=1
  if(ans==Y_test[index]):
    if(ans==1):
      tp+=1
    else:
      tn+=1
  else:
    if(Y_test[index]==1):
      fn+=1
    else:
      fp+=1

Total =  35


# **Model Accuracy Analysis**

In [31]:
print("True Positive = ",tp)
print("True Negative = ",tn)
print("False Positive = ",fp)
print("False Negative = ",fn)

True Positive =  11
True Negative =  7
False Positive =  11
False Negative =  6


In [32]:
accuracy = (tp+tn)/(tp+tn+fp+fn)
sensitivity = tp/(tp+fn)
precision = tp/(tp+fp)
specificity = tn/(tn+fp)

print("accuracy = ",accuracy)
print("sensitivity = ",sensitivity)
print("precision = ",precision)
print("specificity = ",specificity)

accuracy =  0.5142857142857142
sensitivity =  0.6470588235294118
precision =  0.5
specificity =  0.3888888888888889
